### week7 심화과제

RAG 서비스를 제공하는 솔루션에 있는 문서에 따른 질문 추천 모델을 파인튜닝

In [2]:
import json

with open("KorQuAD_v1.0_dev.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
print(f"데이터 개수 : {len(data['data'])}")
print(data['data'][30])

데이터 개수 : 140
{'paragraphs': [{'qas': [{'answers': [{'text': 'TBC', 'answer_start': 208}], 'id': '5782847-0-0', 'question': 'jtbc가 설립하기전에 처음 사용하려고 했던 이름은 무엇인가?'}, {'answers': [{'text': '2009년 7월 22일', 'answer_start': 0}], 'id': '5782847-0-1', 'question': '종편채널 설립에 있어 법적 근거를 부여하는 방송법이 국회를 통과된 날은 언제인가?'}, {'answers': [{'text': '대구방송', 'answer_start': 275}], 'id': '5782847-0-2', 'question': '이름을 jtbc전에 tbc를 사용하려 했으나 이름이 선점하고 있었기 때문인데, 어느 곳이 tbc라는 이름을 사용하고 있었나?'}, {'answers': [{'text': '2010년 11월 30일', 'answer_start': 147}], 'id': '6489801-0-0', 'question': '종합편성채널의 신청서가 접수된 해는?'}, {'answers': [{'text': 'TBC', 'answer_start': 208}], 'id': '6489801-0-1', 'question': 'JTBC 전 종합편성채널에 신청하려 했건 이름은?'}, {'answers': [{'text': 'TBC', 'answer_start': 208}], 'id': '6470395-0-0', 'question': '중앙일보가 종합편성채널의 명칭으로 사용하고자 했던 이름은 무엇인가?'}, {'answers': [{'text': '동양방송', 'answer_start': 195}], 'id': '6470395-0-1', 'question': '중앙일보의 종합편성채널 JTBC라는 명칭에서 TBC는 무엇의 영문 약칭인가?'}], 'context': '2009년 7월 22일에 여러 차례 논란 끝에 

#### ✍️ 학습용 데이터 생성

이미 korQuAD는 QA dataset이기 때문에 해당 데이터에 있는 question을 활용하여 LLM을 사용하지 않고 데이터셋을 만들 수 있습니다.   
RAG에 들어오는 chunk 마다 추천 질문을 만들 것 이기 때문에 여러 질문을 생성하게 하지 않고 한개의 질문을 생성하게 합니다.   
  - 1개의 문서 -> 20개의 chunk -> chunk당 추천 질문 생성 -> 20개의 추천 질문 생성됨 (random 하게 n개 제공)   

In [4]:
import random
import json  # 누락 시 추가
instruction_data = []

for doc in data["data"]:
    paragraphs = doc["paragraphs"]
    valid_paragraphs = [p for p in paragraphs if p["qas"]]  # 질문이 있는 문단만

    if not valid_paragraphs:
        continue  # 질문 없는 문서 제외

    for para in valid_paragraphs:
        context = para["context"].strip()

        for qa in para["qas"]:
            question = qa["question"].strip()

            # instruction-format 구성
            instruction_data.append({
                "instruction": "다음 문단을 읽고 질문을 한가지 만들어 주세요.",
                "input": context,
                "output": question if question.endswith("?") else question + "?"
            })

# 결과 예시 출력
print(f"총 {len(instruction_data)}개 생성됨")
print(json.dumps(instruction_data[0], ensure_ascii=False, indent=2))


총 5774개 생성됨
{
  "instruction": "다음 문단을 읽고 질문을 한가지 만들어 주세요.",
  "input": "1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.",
  "output": "임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?"
}


In [5]:
import random

sampled_data = random.sample(instruction_data, k=1000)

# 예시 출력
print(f"총 {len(sampled_data)}개 샘플링됨")
print(json.dumps(sampled_data[0], ensure_ascii=False, indent=2))

총 1000개 샘플링됨
{
  "instruction": "다음 문단을 읽고 질문을 한가지 만들어 주세요.",
  "input": "하지만 노무현과는 비교되지 않을 정도로 쌓아온 여러 가지 경력, 신뢰감을 주는 이미지, 김대중 정부 말기의 대형 측근 비리 사건은 이회창에게 호재로 작용했다. 그러나 김영삼계열과 이인제, 김윤환 계열의 이탈로 리더십에 타격을 받기도 했다. 그러나 2002년 4월 한나라당 내 김용갑 등 당내 보수파가 그에 대한 공개 지지를 천명하는 등 강경 보수 성향의 인사들로부터 지지를 얻으면서 세력을 만회하는 듯 했다. 2002년 6월 지방선거에서 한나라당은 새천년민주당의 지지도가 높은 호남, 충청 지방을 제외한 대부분의 곳에서 승리를 거둔다. 김대중 당시 대통령의 세 아들도 각종 비리 사건으로 구속되고, 9월에는 정몽준이 월드컵 열기를 타고 대선 출마를 결심하는 등 이회창이 노무현의 초반 돌풍을 극복하고 작은 차이나마 꾸준히 앞서나가는 모습을 보인다.",
  "output": "2002년 9월 월드컵 열기에 힘입어 대선 출마를 결심한 사람은?"
}


In [6]:
with open("corpus.json", "w", encoding="utf-8") as f:
    json.dump(sampled_data, f, ensure_ascii=False, indent=2)

### 학습

In [1]:
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import torch
from tqdm import tqdm
import os

In [2]:
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

In [3]:
with open("corpus.json", "r", encoding="utf-8") as f:
    corpus = json.load(f)

In [4]:
train_data, temp_data = train_test_split(corpus, test_size=0.1, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=1/2, random_state=42)

In [5]:
dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data),
    "test": Dataset.from_list(test_data),
})

print("** Dataset size after tokenization and grouping:")
print(dataset)

** Dataset size after tokenization and grouping:
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 50
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 50
    })
})


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [7]:
def format_and_tokenize(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""

    tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(format_and_tokenize, batched=False)

print("finish preparing data")

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

finish preparing data


In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
base_model.config.use_cache = False
base_model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [9]:
for name, module in base_model.named_modules():
    if 'attn' in name.lower():
        print(name)

transformer.h.0.attn
transformer.h.0.attn.attention
transformer.h.0.attn.attention.rotary
transformer.h.0.attn.attention.k_proj
transformer.h.0.attn.attention.v_proj
transformer.h.0.attn.attention.q_proj
transformer.h.0.attn.attention.out_proj
transformer.h.1.attn
transformer.h.1.attn.attention
transformer.h.1.attn.attention.rotary
transformer.h.1.attn.attention.k_proj
transformer.h.1.attn.attention.v_proj
transformer.h.1.attn.attention.q_proj
transformer.h.1.attn.attention.out_proj
transformer.h.2.attn
transformer.h.2.attn.attention
transformer.h.2.attn.attention.rotary
transformer.h.2.attn.attention.k_proj
transformer.h.2.attn.attention.v_proj
transformer.h.2.attn.attention.q_proj
transformer.h.2.attn.attention.out_proj
transformer.h.3.attn
transformer.h.3.attn.attention
transformer.h.3.attn.attention.rotary
transformer.h.3.attn.attention.k_proj
transformer.h.3.attn.attention.v_proj
transformer.h.3.attn.attention.q_proj
transformer.h.3.attn.attention.out_proj
transformer.h.4.attn
tra

In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"]
)

In [11]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 3,993,600 || all params: 2,409,320,960 || trainable%: 0.1658


In [12]:
# ✅ TrainingArguments
training_args = TrainingArguments(
    output_dir="./exaone-lora-finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="no",
    save_total_limit=1,
    run_name="exaone-lora-tuning",
    learning_rate=1e-5,
    fp16=True,
    label_names=["labels"],
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

In [14]:
def make_prompt(example):
    return f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:"""

In [15]:
def eval_model(model):
    model.eval()
    results = []
    for example in tqdm(dataset["test"]):
        prompt = make_prompt(example)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id)
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        model_output = decoded_output.split("### Response:")[-1].strip()

        results.append({
            "input": example["input"],
            "expected_output": example["output"],
            "model_output": model_output
        })
    return results


In [16]:
# init_result = eval_model(model)

# with open("init_outputs.json", "w", encoding="utf-8") as f:
#     json.dump(init_result, f, ensure_ascii=False, indent=2)

In [17]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cyooon-kim (cyooon-kim-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.858000,1.417500
2,1.424700,1.396953
3,1.410500,1.391738


TrainOutput(global_step=1350, training_loss=1.8977506510416666, metrics={'train_runtime': 1542.7607, 'train_samples_per_second': 1.75, 'train_steps_per_second': 0.875, 'total_flos': 1.7809544577024e+16, 'train_loss': 1.8977506510416666, 'epoch': 3.0})

In [18]:
after_result = eval_model(trainer.model)

with open("after_outputs.json", "w", encoding="utf-8") as f:
    json.dump(after_result, f, ensure_ascii=False, indent=2)

100%|██████████| 50/50 [01:25<00:00,  1.71s/it]


In [19]:
trainer.save_model()